In [41]:
import pandas as pd
from sqlalchemy import create_engine

connect = create_engine("postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml")

In [70]:
posts = pd.read_sql('''SELECT * 
                 FROM post_text_df p
                 ''', connect, index_col='post_id')

In [71]:
users = pd.read_sql('''SELECT * 
                 FROM user_data u
                 ''', connect, index_col='user_id')

In [75]:
import time
start_time = time.time()
df = pd.read_sql('''select t.*
                        from (select f.*
                                ,row_number() over (partition by user_id order by random()) as seqnum
                                ,count(*) over () as cnt
                                ,count(*) over (partition by user_id) as cc_cnt
                              from feed_data f
                              where action = 'view'
                             ) t
                        where seqnum <= (cc_cnt*cc_cnt)/(cnt*0.001)''', connect)
print("--- %s sec ---" % (time.time() - start_time))
#cnt*0.0001 - 4794602 rows - 818 sec
#cnt*0.001 - 406541 rows - 284 sec

--- 1238.415589094162 sec ---


In [76]:
df.info()
#                         inner join user_data u
#                                 on u.user_id = t.user_id
#                         inner join post_text_df p
#                                 on p.post_id = t.post_id

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406541 entries, 0 to 406540
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   timestamp  406541 non-null  datetime64[ns]
 1   user_id    406541 non-null  int64         
 2   post_id    406541 non-null  int64         
 3   action     406541 non-null  object        
 4   target     406541 non-null  int64         
 5   seqnum     406541 non-null  int64         
 6   cnt        406541 non-null  int64         
 7   cc_cnt     406541 non-null  int64         
dtypes: datetime64[ns](1), int64(6), object(1)
memory usage: 24.8+ MB


In [77]:
df = df.merge(users, how='inner', on='user_id')
df = df.merge(posts, how='inner', on='post_id')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406541 entries, 0 to 406540
Data columns (total 17 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   timestamp  406541 non-null  datetime64[ns]
 1   user_id    406541 non-null  int64         
 2   post_id    406541 non-null  int64         
 3   action     406541 non-null  object        
 4   target     406541 non-null  int64         
 5   seqnum     406541 non-null  int64         
 6   cnt        406541 non-null  int64         
 7   cc_cnt     406541 non-null  int64         
 8   gender     406541 non-null  int64         
 9   age        406541 non-null  int64         
 10  country    406541 non-null  object        
 11  city       406541 non-null  object        
 12  exp_group  406541 non-null  int64         
 13  os         406541 non-null  object        
 14  source     406541 non-null  object        
 15  text       406541 non-null  object        
 16  topic      406541 no

In [78]:
# users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163205 entries, 200 to 168552
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   gender     163205 non-null  int64 
 1   age        163205 non-null  int64 
 2   country    163205 non-null  object
 3   city       163205 non-null  object
 4   exp_group  163205 non-null  int64 
 5   os         163205 non-null  object
 6   source     163205 non-null  object
dtypes: int64(3), object(4)
memory usage: 10.0+ MB


In [73]:
# user_cat_cols = users.loc[:,users.dtypes==object].columns.to_list()
# user_num_columns = users.loc[:,users.dtypes!=object].columns.to_list()

['gender', 'age', 'exp_group']

In [97]:
# from category_encoders.one_hot import OneHotEncoder
# enc = OneHotEncoder(handle_unknown='ignore')
# new_users = enc.fit_transform(users)

In [102]:
# new_users

,gender,age,country_1,country_2,country_3,country_4,country_5,country_6,country_7,country_8,...,city_3911,city_3912,city_3913,city_3914,city_3915,exp_group,os_1,os_2,source_1,source_2
user_id,,,,,,,,,,,,,,,,,,,,,
200,1,34,1,0,0,0,0,0,0,0,...,0,0,0,0,0,3,1,0,1,0
201,0,37,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
202,1,17,1,0,0,0,0,0,0,0,...,0,0,0,0,0,4,1,0,1,0
203,0,18,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,1,0
204,0,36,1,0,0,0,0,0,0,0,...,0,0,0,0,0,3,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168548,0,36,1,0,0,0,0,0,0,0,...,0,0,0,0,0,4,1,0,0,1
168549,0,18,1,0,0,0,0,0,0,0,...,0,0,0,0,0,2,1,0,0,1
168550,1,41,1,0,0,0,0,0,0,0,...,0,0,0,0,0,4,1,0,0,1


In [ ]:
# pairs_of_hyperparams = [
#                             [0.1,20],
#                             [0.1,50],
#                             [0.2,20],
#                             [0.2,50],
#                             [0.3,20],
#                             [0.3,50],
#                        ]

# from sklearn.cluster import DBSCAN

# fig = plt.figure()
# fig.set_size_inches(20, 50)

# for i, s in enumerate(pairs_of_hyperparams):
    
#     X = dataset[["x1", "x2"]]
    
#     dbscan = DBSCAN(eps=s[0], min_samples=s[1]).fit(X)
#     ax_ = fig.add_subplot(5, 2, i+1)
    
#     colors = ["black", "#FF5533",
#               "#00B050", "orange",
#               "blue", "purple"]
    
#     y = dbscan.labels_
#     labels = sorted(list(set(dbscan.labels_)))
    
#     if len(labels)==1 and y[0]==0:
#         colors = {0:"#FF5533"}
#     else:
#         colors = dict(zip(labels, colors))

    
#     for i, c in zip(labels, colors.values()):
        
#         idx = np.where(y == i)
        
#         plt.scatter(
#             X.values[idx, 0],
#             X.values[idx, 1],
#             c=c,
#             s=20,
#             edgecolor="k",
#         )
#     plt.xlim(x1_min, x1_max)
#     plt.ylim(x2_min, x2_max)
#     plt.xlabel("x")
#     plt.ylabel("y")
    
#     if len(y)==1 and y[0]==0:
#         n_clusters = 1
#     else:
#         n_clusters = len(set(y))-1
    
#     plt.title(f"Epsilon = {s[0]}\n min_samples = {s[1]} \n with {n_clusters} clusters")
    
# fig.tight_layout()
# plt.show()

In [ ]:
from sklearn.cluster import KMeans
klastMeans(random_state = 1, n_clusters=4)
# pipe_dt.fit(X_train, y_train)

In [47]:
data = df.copy()
cln_drop = ['action', 'cnt']
data = data.drop(cln_drop, axis=1)

In [103]:
data.head()

,target,seqnum,gender,age,country,city,exp_group,os,source,text,topic
0,0,1,1,34,Russia,Degtyarsk,3,Android,ads,Don’t be deceived by cases down 6% for the wee...,covid
1,0,3,0,34,Russia,Barnaul,1,Android,ads,Don’t be deceived by cases down 6% for the wee...,covid
2,1,4,0,16,Russia,Solikamsk,2,iOS,ads,Don’t be deceived by cases down 6% for the wee...,covid
3,0,2,0,41,Russia,Mtsensk,3,iOS,ads,Don’t be deceived by cases down 6% for the wee...,covid
4,0,5,0,33,Ukraine,Sniatyn,4,Android,ads,Don’t be deceived by cases down 6% for the wee...,covid


In [48]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf = TfidfVectorizer(use_idf=True, smooth_idf=True, sublinear_tf=False)
# tfidf.fit(posts['text'])

In [49]:
# df = pd.DataFrame(tfidf.transform(posts['text']).todense(),
#                   columns=tfidf.get_feature_names())

In [50]:
# holdout_idx = int(len(df)*0.9)
# X_train =  pd.DataFrame(posts['text'].iloc[:holdout_idx])
# X_test =   pd.DataFrame(posts['text'].iloc[holdout_idx:])


In [51]:
# def vectorize(train, test, vectorizer_params):
#     # Join each row by space and apply an TF-IDF vectorizer to each row
#     vectorizer = TfidfVectorizer(**vectorizer_params)
#     func = lambda row: ' '.join(row)
#     train_s = train.apply(func, axis=1).tolist()
#     train_sparse = vectorizer.fit_transform(train_s)
    
#     test_s = test.apply(func, axis=1).tolist()
#     test_sparse = vectorizer.transform(test_s)
    
#     return train_sparse, test_sparse, vectorizer

In [52]:
# vectorizer_params = {
#     'ngram_range': (1, 1), 
#     'max_features': 50000, 
#     'tokenizer': lambda s: s.split()
# }
# X_train_sparse, X_test_sparse, vectorizer = vectorize(X_train, X_test, vectorizer_params)
# #X_full_sparse, X_test_sparse, _ = vectorize(X_full, X_test_sites, vectorizer_params)

# print(X_train_sparse.shape, X_test_sparse.shape)

In [53]:
# def stack_features(*matrices):
#     # Stack matrices into a single sparse matrix
#     return csr_matrix(hstack(matrices))

In [54]:
text_columns = ['text']
categorical_columns = data.loc[:,data.dtypes==object].columns.to_list()
categorical_columns.remove(*text_columns)
categorical_columns

['country', 'city', 'os', 'source', 'topic']

In [55]:
X = data.drop('target', axis=1)
y = data['target']

In [56]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=0,
                                                    shuffle=True,
                                                    test_size=0.2)

In [62]:
from catboost import CatBoostClassifier

### Your code is here
catboost = CatBoostClassifier(random_state=1, auto_class_weights='Balanced')

catboost.fit(X_train, y_train, cat_features=categorical_columns, text_features=text_columns, verbose=False, plot=True)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [63]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [64]:
test_preds = catboost.predict(X_test)
print(accuracy_score(y_test, test_preds))
print(classification_report(y_test, test_preds, digits=3))

0.6304099177212855
              precision    recall  f1-score   support

           0      0.920     0.635     0.751     71515
           1      0.183     0.597     0.280      9794

    accuracy                          0.630     81309
   macro avg      0.551     0.616     0.516     81309
weighted avg      0.831     0.630     0.695     81309



In [65]:
catboost.save_model('catboost_model',
                           format="cbm")


In [123]:
# def rec_list(user_id, time):
from datetime import datetime
user_id = 200
#pred_df = posts 
pred_df = users[users.index == user_id].copy()
pred_df['tmp'] = 1
posts['tmp'] = 1

pred_df = pd.merge(pred_df, posts, on=['tmp'])
posts = posts.drop('tmp', axis=1)
pred_df['tmp'] = datetime.now()

# pred_df = posts.join(pred_df, how='outer')

In [124]:
pred_df.head()

,gender,age,country,city,exp_group,os,source,tmp,text,topic
0,1,34,Russia,Degtyarsk,3,Android,ads,2022-08-11 21:04:19.282054,UK economy facing major risks\n\nThe UK manufa...,business
1,1,34,Russia,Degtyarsk,3,Android,ads,2022-08-11 21:04:19.282054,Aids and climate top Davos agenda\n\nClimate c...,business
2,1,34,Russia,Degtyarsk,3,Android,ads,2022-08-11 21:04:19.282054,Asian quake hits European shares\n\nShares in ...,business
3,1,34,Russia,Degtyarsk,3,Android,ads,2022-08-11 21:04:19.282054,India power shares jump on debut\n\nShares in ...,business
4,1,34,Russia,Degtyarsk,3,Android,ads,2022-08-11 21:04:19.282054,Lacroix label bought by US firm\n\nLuxury good...,business


In [119]:
real_preds = catboost.predict(pred_df)


CatBoostError: Invalid cat_features[4] = 9 value: index must be < 9.

In [ ]:
real_preds.head()